In [ ]:
# Update sklearn to prevent version mismatches
!pip install sklearn --upgrade

In [ ]:
# install joblib. This will be used to save your model. 
# Restart your kernel after installing 
!pip install joblib

In [1]:
import pandas as pd

# Read the CSV and Perform Basic Data Cleaning

In [2]:
df = pd.read_csv("exoplanet_data.csv")
# Drop the null columns where all values are null
df = df.dropna(axis='columns', how='all')
# Drop the null rows
df = df.dropna()
df.head()

,koi_disposition,koi_fpflag_nt,koi_fpflag_ss,koi_fpflag_co,koi_fpflag_ec,koi_period,koi_period_err1,koi_period_err2,koi_time0bk,koi_time0bk_err1,...,koi_steff_err2,koi_slogg,koi_slogg_err1,koi_slogg_err2,koi_srad,koi_srad_err1,koi_srad_err2,ra,dec,koi_kepmag
0,CONFIRMED,0,0,0,0,54.418383,2.479000e-04,-2.479000e-04,162.513840,0.003520,...,-81,4.467,0.064,-0.096,0.927,0.105,-0.061,291.93423,48.141651,15.347
1,FALSE POSITIVE,0,1,0,0,19.899140,1.490000e-05,-1.490000e-05,175.850252,0.000581,...,-176,4.544,0.044,-0.176,0.868,0.233,-0.078,297.00482,48.134129,15.436
2,FALSE POSITIVE,0,1,0,0,1.736952,2.630000e-07,-2.630000e-07,170.307565,0.000115,...,-174,4.564,0.053,-0.168,0.791,0.201,-0.067,285.53461,48.285210,15.597
3,CONFIRMED,0,0,0,0,2.525592,3.760000e-06,-3.760000e-06,171.595550,0.001130,...,-211,4.438,0.070,-0.210,1.046,0.334,-0.133,288.75488,48.226200,15.509
4,CONFIRMED,0,0,0,0,4.134435,1.050000e-05,-1.050000e-05,172.979370,0.001900,...,-232,4.486,0.054,-0.229,0.972,0.315,-0.105,296.28613,48.224670,15.714


# Select your features (columns)

In [32]:
# Set features. This will also be used as your x values.
selected_features = df[['koi_tce_plnt_num', 'koi_srad', 'koi_insol', 'koi_duration', 'koi_depth']]

,koi_tce_plnt_num,koi_srad,koi_insol,koi_duration,koi_depth
0,2,0.927,9.11,4.50700,874.8
1,1,0.868,39.30,1.78220,10829.0
2,1,0.791,891.96,2.40641,8079.2
3,1,1.046,926.16,1.65450,603.3
4,2,0.972,427.65,3.14020,686.0
...,...,...,...,...,...
6986,1,1.088,176.40,4.80600,87.7
6987,1,0.903,4500.53,3.22210,1579.2
6988,1,1.031,1585.81,3.11400,48.5
6989,1,1.041,5713.41,0.86500,103.6


# Create a Train Test Split

Use `koi_disposition` for the y values

In [106]:
from sklearn.model_selection import train_test_split
x_data = selected_features
y_data = df[['koi_disposition']]
y_data['koi_disposition'] = pd.Categorical(y_data['koi_disposition'])
y_data['koi_disposition'] = y_data['koi_disposition'].cat.codes

x_train, x_test, y_train, y_test = train_test_split(x_data, y_data, test_size=0.8, random_state=0)

<ipython-input-106-7e88d454c046>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  y_data['koi_disposition'] = pd.Categorical(y_data['koi_disposition'])
<ipython-input-106-7e88d454c046>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  y_data['koi_disposition'] = y_data['koi_disposition'].cat.codes


In [107]:
x_train.head()

,koi_tce_plnt_num,koi_srad,koi_insol,koi_duration,koi_depth
3436,1,0.596,30.83,2.4460,199.3
4256,1,1.223,3837.83,6.3373,401.3
6764,1,0.909,295.17,1.9980,99.4
6494,1,1.393,95.35,6.8520,57.0
6264,1,0.812,12.88,2.2080,302.0


# Pre-processing

Scale the data using the MinMaxScaler and perform some feature selection

In [108]:
# Scale your data
from sklearn import linear_model
from sklearn import preprocessing  
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()            

scaler.fit(x_train)
x_train_scaled = scaler.transform(x_train)
scaler.fit(x_test)
x_test_scaled = scaler.transform(x_test)

# Train the Model



In [109]:
from sklearn import linear_model
reg = linear_model.LinearRegression()
reg.fit(x_train, y_train)
print(f"Training Data Score: {reg.score(x_train_scaled, y_train)}")
print(f"Testing Data Score: {reg.score(x_test_scaled, y_test)}")

Training Data Score: 0.004836716287051512
Testing Data Score: 0.0006551907096109533


# Hyperparameter Tuning

Use `GridSearchCV` to tune the model's parameters

In [113]:
# Create the GridSearchCV model
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestRegressor

tree_param_grid = { 'min_samples_split': list((3,6,9)),'n_estimators':list((10,50,100))}
grid = GridSearchCV(RandomForestRegressor(),param_grid=tree_param_grid, cv=5)

In [114]:
# Train the model with GridSearch
grid.fit(x_train, y_train['koi_disposition'].ravel())


GridSearchCV(cv=5, estimator=RandomForestRegressor(),
             param_grid={'min_samples_split': [3, 6, 9],
                         'n_estimators': [10, 50, 100]})

In [115]:
print(grid.best_params_)
print(grid.best_score_)

{'min_samples_split': 9, 'n_estimators': 50}
0.2689375992870119


# Save the Model

In [117]:
# save your model by updating "your_name" with your name
# and "your_model" with your model variable
# be sure to turn this in to BCS
# if joblib fails to import, try running the command to install in terminal/git-bash
import joblib
filename = 'tanlin_hung.sav'
joblib.dump(reg, filename)

['tanlin_hung.sav']